## **Task 3: Preprocessing data**
(Edit from Quan_Luu_Preprocessing_data branch)
<hr/>

In [14]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pandas as pd
from tqdm import tqdm

### *1. Read raw data from mongoDB Local*

In [15]:
# Step 1: Read raw data
# Read data from MongoDB and save it in a list
# Read 50000 data from MongoDB and save it in a dataframe
def read_raw_data(collection, x, y):
    data_list = []
    total_documents = y - x
    with tqdm(total=total_documents, desc='Getting Documents') as pbar:
        for document in collection.find({"crawl_id": {"$gte": x, "$lt": y}}):
            del document['_id']
            data_list.append(document)
            pbar.update(1)    

    df = pd.DataFrame(data_list)
    return df

### *2. Drop unused columns*

#### 2.1. Drop unused columns

In [16]:
# Step 2: Drop unused columns
def drop_unused_columns(df):
       # List of fixed columns to keep
       used_columns = ['itemid', 'shopid', 'name', 'stock', 'sold', 'historical_sold',
              'liked_count', 'cmt_count', 'item_status', 'price', 'price_min',
              'price_max', 'price_min_before_discount', 'price_max_before_discount',
              'price_before_discount', 'tier_variations', 'item_rating',
              'show_free_shipping', 'shop_location', 'is_on_flash_sale', 'shop_name',
              'shop_rating', 'global_sold_count', 'flash_sale_stock', 'crawl_time',
              'crawl_id'] 

       unused_columns = [col for col in df.columns if col not in used_columns]

       # Dropping columns that are not in the fixed list
       df = df.drop(columns=unused_columns)

       return df

#### 2.2. Column Information
Mô tả các cột dữ liệu và các kiểu dữ liệu mong muốn

|STT| Tên cột    | Mô tả| Kiểu dữ liệu mong muốn |
|---|:-----------:|:-------------:|:---|
|1| itemid| Id của sản phẩm| String|
|2| shopid| ID của shop| string|
|3| stock| Số lượng sản phẩm trong kho| int |
|4| sold| Số lượng sản phẩm đã bán| int|
|5| liked_count| Số lượt thích sản phẩm| int|
|6| cmt_count| Số lượt bình luận| int|
|7| item_status| trạng thái sản phẩm| bool|
|8| price| giá sản phẩm| float|
|9| price_min|giá tối thiểu| float|
|10| price_max| giá tối đa| float|
|11| price_min_before_discount| giá tối thiểu trước discount| float|
|12| price_max_before_discount| giá tối đa trước discount| float|
|13| price_before_discount| giá trước discount| string|
|14| tier_variations| kích thước, màu sắc và các đặc tính khác của sản phẩm| dict|
|15| item_rating| các đánh giá của sản phẩm(bao gồm đánh giá trung bình và số đánh giá của từng sao)| dict|
|16| show_free_shipping| shop có free ship hay không| bool|
|17| shop_location|vị trí của shop| string|
|18| is_on_flash_sale| sản phẩm có đang sale hay không| bool|
|19| shop_name| tên shop| string|
|20| shop_rating| đánh giá của shop| float|
|21| global_sold_count| số lượng sản phẩm được bán trên thế giới trên shoppe| int|
|22| flash_sale_stock| số lượng sản phẩm còn lại trong đợt flash sale| string|
|23| crawl_time| thời điểm crawl data| date time|
|24| historical _sold| Số lượng đã bán trong quá khứ| int|
|25| name| tên sản phẩm| string|


### *3. Edit data columns*
Có nhiều cột có kiểu dữ liệu không hợp lý nên chuyển chúng về kiểu dữ liệu phù hợp với việc phân tích

#### 3.1. Edit Price
các cột giá cả bị lớn hơn 100000 lần

In [17]:
# Step 3: Edit data columns
## Step 3.1: Edit price
def edit_price(df):
    price_columns = ['price','price_min','price_max','price_min_before_discount','price_max_before_discount','price_before_discount']
    df[price_columns]=df[price_columns]/100000
    return df

#### 3.2. Convert data type

In [18]:
## Step 3.2: Convert data type
def convert_data_type(df):
    int_columns = ['stock','sold','historical_sold','liked_count','cmt_count','global_sold_count','flash_sale_stock','crawl_id']
    df['itemid']=df['itemid'].astype('str')
    df['shopid']=df['shopid'].astype('str')
    df[int_columns]=df[int_columns].astype('int32')
    return df

#### 3.3. Edit some object columns

##### 3.3.1. Tier variations columns

In [19]:
## Step 3.3: Edit some object columns
# hàm lấy ra màu
colors = ["ĐỎ","VÀNG","TRẮNG","ĐEN","HỒNG","TÍM","CAM","NÂU","XÁM","LAM",'GHI','XANH','BE','THAN','RÊU','CHÌ','NGẪU','TIÊU']

def is_color(list_color:str):
    list_return = []
    #item = item.split('.')[0]
    for item in list_color:
        item=item.replace(',',' ')
        item=item.replace('XDƯƠNG','XANH')
        item=item.replace('XANHĐÁ-Mũ','XANH')
        split_item = item.split(' ')
        for color in split_item:
            if '-' in color:
                color.replace(" ","")
                a = color.split('-')
                for cl in a:
                    if cl.upper() in colors:
                        list_return.append(cl)
            else:
                if color.upper() in colors:
                    list_return.append(color)
        if len(list_return) == 0:
            return 'NGẪU NHIÊN'
        return list_return
    
def get_options_size(columns):
    if len(columns) > 1:
        return columns[1]['options']
    return ['No size']

def get_options_color(columns):
        return columns[0]['options']

def explore_color(array):
    if type(array) == list:
        return ", ".join(array)
    else:
        return array

### Step 3.3.1 : Edit tier variations columns
def edit_tier_variations_columns(df):
    # color
    df['color']=df['tier_variations'].apply(get_options_color)
    df['color']=df['color'].apply(is_color)
    df['color']=df['color'].apply(explore_color)
    # size
    df['size']=df['tier_variations'].apply(get_options_size)
    # drop tier_variations columns
    df.drop('tier_variations',axis=1,inplace=True)
    return df

##### 3.3.2. Item rating columns

In [20]:
### Step 3.3.2: Edit item rating columns
# lấy rating
def get_rating_star(item):
    return item['rating_star']
def get_rating_count(item):
    return item['rating_count']
def get_rcount_with_image(item):
    return item['rcount_with_image']
def get_rcount_with_context(item):
    return item['rcount_with_context']

def edit_item_rating_columns(df):
    # init new column
    df['rating_star'] = df['item_rating'].apply(get_rating_star)
    df['rating_count'] = df['item_rating'].apply(get_rating_count)
    df['rcount_with_image'] = df['item_rating'].apply(get_rcount_with_image)
    df['rcount_with_context'] = df['item_rating'].apply(get_rcount_with_context)
    return df

### *4. Pipeline for preprocessing data*

In [21]:
def preprocessing_pipeline(collection, x, num_documents):
    y = x + 20000
    if y > num_documents:
        y = num_documents
    
    # Step 1: Read raw data
    df = read_raw_data(collection, x, y)
    # Step 2: Drop unused columns
    df = drop_unused_columns(df)
    # Step 3: Edit data columns
    ## Step 3.1: Edit price
    df = edit_price(df)
    ## Step 3.2: Convert data type
    df = convert_data_type(df)
    ## Step 3.3: Edit some object columns
    ### Step 3.3.1 : Edit tier variations columns
    df = edit_tier_variations_columns(df)
    ### Step 3.3.2: Edit item rating columns
    df = edit_item_rating_columns(df)

    return df

### *5. Store preprocessing data*

In [22]:
client = MongoClient('mongodb://localhost:27017/')
db = client['Task2_Database']
collection = db['Shopee_Full_Data']

# Count the number of documents in the collection
num_documents_local = collection.count_documents({}) 
print("Number of documents in the collection:", num_documents_local)

Number of documents in the collection: 441322


In [25]:
client_p = MongoClient('mongodb://localhost:27017/')
db_p = client_p ['Task2_Database']
collection_p = db_p['Shopee_Full_Preprocessing_Data']
num_documents_server = collection_p.count_documents({})
print("Number of documents in the collection:", num_documents_server)

Number of documents in the collection: 441322


In [24]:
df = preprocessing_pipeline(collection, num_documents_server, num_documents_local)

dict_df = df.to_dict('records')
result = collection_p.insert_many(dict_df)
print ('=> Inserted document IDs successfuly:', result.inserted_ids)

password = "qthls2023"
uri = "mongodb+srv://shopee_database_1:{}@cluster0.1wsiirv.mongodb.net/?retryWrites=true&w=majority".format(password)
client_server = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client_server.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

db_server = client_server['Task2_Database']
collection_server = db_server['Shopee_Full_Preprocessing_Data']
dict_df = df.to_dict('records')
result = collection_server.insert_many(dict_df)
print ('=> Inserted document IDs successfuly:', result.inserted_ids)

Getting Documents: 100%|██████████| 8270/8270 [00:14<00:00, 568.40it/s]


=> Inserted document IDs successfuly: [ObjectId('656614e5e010dfc9ecb63c36'), ObjectId('656614e5e010dfc9ecb63c37'), ObjectId('656614e5e010dfc9ecb63c38'), ObjectId('656614e5e010dfc9ecb63c39'), ObjectId('656614e5e010dfc9ecb63c3a'), ObjectId('656614e5e010dfc9ecb63c3b'), ObjectId('656614e5e010dfc9ecb63c3c'), ObjectId('656614e5e010dfc9ecb63c3d'), ObjectId('656614e5e010dfc9ecb63c3e'), ObjectId('656614e5e010dfc9ecb63c3f'), ObjectId('656614e5e010dfc9ecb63c40'), ObjectId('656614e5e010dfc9ecb63c41'), ObjectId('656614e5e010dfc9ecb63c42'), ObjectId('656614e5e010dfc9ecb63c43'), ObjectId('656614e5e010dfc9ecb63c44'), ObjectId('656614e5e010dfc9ecb63c45'), ObjectId('656614e5e010dfc9ecb63c46'), ObjectId('656614e5e010dfc9ecb63c47'), ObjectId('656614e5e010dfc9ecb63c48'), ObjectId('656614e5e010dfc9ecb63c49'), ObjectId('656614e5e010dfc9ecb63c4a'), ObjectId('656614e5e010dfc9ecb63c4b'), ObjectId('656614e5e010dfc9ecb63c4c'), ObjectId('656614e5e010dfc9ecb63c4d'), ObjectId('656614e5e010dfc9ecb63c4e'), ObjectId('6